In [1]:
from google.cloud import bigquery
import pandas as pd
import geopandas as gpd
import pydata_google_auth
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    # but you can do this if you forward port 8080 over to the machine
    auth_local_webserver=True,
)
napa_fips = '06055'
sonoma_fips = '06097'

client = bigquery.Client(credentials=credentials)
sql_query = """
SELECT x.Structure_StdAddressLine, x.Structure_StdCity, x.STructure_StdState, x.Structure_StdPostcode, y.* FROM 
`clgx-algorithm-proc-dev-eddd.spatial_FFH_RISKMETER_TESTING.tmp_chuck_strct_unique_ffh_geom` y INNER JOIN
`clgx-algorithm-proc-dev-eddd.spatial_FFH_sbx.FFH_DH_Model_Analysis` x
ON y.STRCT_ID = x.STRUCT_ID
WHERE fips in ('06055','06097')
"""

df = client.query(sql_query).to_dataframe()

Reauthentication required.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=UWDSWaIOSLc5jNYH4dW8LPPVGrfMau&access_type=offline


In [2]:
df.head()

,Structure_StdAddressLine,Structure_StdCity,STructure_StdState,Structure_StdPostcode,STRCT_ID,FIPS,LAG_FT,HAG_FT,FFH_FT,geom
0,153 GAMAY CIR,YOUNTVILLE,CA,94599,321303598,06055,84.4,85.3,2.9,"POLYGON((-122.35535829637 38.3985306554996, -1..."
1,1870 HERITAGE WAY,YOUNTVILLE,CA,94599,321304328,06055,89.3,90.5,-0.2,"POLYGON((-122.356370367444 38.4046905044651, -..."
2,20088 EL RANCHO WAY,MONTE RIO,CA,95462,327944705,06097,45.5,46.9,1.1,"POLYGON((-123.01363046383 38.4707243507015, -1..."
3,1927 OAK CIR,YOUNTVILLE,CA,94599,321302941,06055,88.8,90.8,1.0,"POLYGON((-122.356718201554 38.4017974666649, -..."
4,105 CLARET CIR,YOUNTVILLE,CA,94599,321316213,06055,85.6,86.4,1.5,"POLYGON((-122.355201333893 38.3975167769124, -..."


In [3]:
from shapely import wkt

In [4]:
df['geometry'] = df['geom'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, crs="EPSG:4326",geometry='geometry')
gdf['centroid'] = gdf.geometry.centroid

/tmp/ipykernel_14215/3617946795.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


In [5]:
from shapely.geometry import Point
import geopy.distance
glass_fire_lat_lon = [38.56295, -122.49745]
gdf['distance'] = gdf['centroid'].apply(lambda x:geopy.distance.geodesic((x.coords[0][1],x.coords[0][0]),tuple(glass_fire_lat_lon)))

In [6]:
gdf['distance'].apply(lambda x: x.km).describe()

count    324440.000000
mean         30.696006
std          10.543085
min           0.142701
25%          25.098293
50%          30.877213
75%          35.776877
max          92.094640
Name: distance, dtype: float64

In [7]:
gdf['within_10'] = gdf['distance'].apply(lambda x: x.km)<=10

In [8]:
gdf['within_10'].sum()

12902

In [9]:
# !pip install geopy

In [11]:
gdf.columns

Index(['Structure_StdAddressLine', 'Structure_StdCity', 'STructure_StdState',
       'Structure_StdPostcode', 'STRCT_ID', 'FIPS', 'LAG_FT', 'HAG_FT',
       'FFH_FT', 'geom', 'geometry', 'centroid', 'distance', 'within_10'],
      dtype='object')

In [12]:
gdf_save = gdf.loc[gdf['within_10']].reset_index(drop=True).to_crs('EPSG:3857')

In [13]:
gdf_save['distance'] = gdf_save['distance'].apply(lambda x: x.km)

In [14]:
gdf_save = gdf_save.rename(columns=dict(zip(['Structure_StdAddressLine', 'Structure_StdCity', 'STructure_StdState',
       'Structure_StdPostcode'],[x.lower().replace('structure_std','') for x in ['Structure_StdAddressLine', 'Structure_StdCity', 'STructure_StdState',
       'Structure_StdPostcode']])))
gdf_save.head()

,addressline,city,state,postcode,STRCT_ID,FIPS,LAG_FT,HAG_FT,FFH_FT,geom,geometry,centroid,distance,within_10
0,850 SANITARIUM RD,DEER PARK,CA,94576,321322448,06055,737.2,746.5,1.0,"POLYGON((-122.478484854708 38.5506102352289, -...","POLYGON ((-13634242.567 4657503.252, -13634245...",POINT (-122.47831 38.55054),2.163731,True
1,6275 POPE VALLEY RD,POPE VALLEY,CA,94567,321323095,06055,786.0,787.5,2.5,"POLYGON((-122.442825951532 38.6230340647152, -...","POLYGON ((-13630273.036 4667817.384, -13630281...",POINT (-122.44286 38.62300),8.188314,True
2,1091 DEER PARK RD,DEER PARK,CA,94576,321318798,06055,920.4,924.3,0.5,"POLYGON((-122.47149729836 38.5558125492606, -1...","POLYGON ((-13633464.716 4658243.785, -13633479...",POINT (-122.47154 38.55571),2.396930,True
3,6299 POPE VALLEY RD,POPE VALLEY,CA,94567,321323097,06055,900.3,908.1,1.0,"POLYGON((-122.449858425243 38.6222300663849, -...","POLYGON ((-13631055.888 4667702.827, -13631055...",POINT (-122.44996 38.62219),7.769890,True
4,7161 POPE VALLEY RD,POPE VALLEY,CA,94567,321322836,06055,723.6,725.4,1.0,"POLYGON((-122.465845050311 38.648421805859, -1...","POLYGON ((-13632835.511 4671435.407, -13632831...",POINT (-122.46584 38.64845),9.882574,True


In [15]:
gdf_save.columns

Index(['addressline', 'city', 'state', 'postcode', 'STRCT_ID', 'FIPS',
       'LAG_FT', 'HAG_FT', 'FFH_FT', 'geom', 'geometry', 'centroid',
       'distance', 'within_10'],
      dtype='object')

In [16]:
gdf_save[['addressline', 'city', 'state', 'postcode', 'STRCT_ID', 'FIPS', 'LAG_FT', 'HAG_FT',
       'FFH_FT', 'distance','geometry']].to_file('sample_counties.shp')

/tmp/ipykernel_14215/3239955059.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_save[['addressline', 'city', 'state', 'postcode', 'STRCT_ID', 'FIPS', 'LAG_FT', 'HAG_FT',


In [17]:
!zip sample_counties.zip sample_counties*

updating: sample_counties.cpg (stored 0%)
updating: sample_counties.dbf (deflated 94%)
updating: sample_counties.prj (deflated 42%)
updating: sample_counties.shp (deflated 45%)
updating: sample_counties.shx (deflated 57%)
